In [17]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
from tensorflow.keras.datasets import imdb
from nltk.tokenize import word_tokenize

import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
import numpy as np

import sqlite3

In [18]:
ckey = "DnAPBR25Wo9g2mAbbQe7jNnJf"
csecret = "XBSR3bwR9Eadk1ykDb9ScKZKvm5p2CDjf69mYnsR8RuBTHIiYh"

In [19]:
atoken = "1179559654700941312-UUh5kKPHp8sA5e6YYFwl7NpBNzz7C3"
asecret = "4JeWZ2lIMcH0rvbD5eWsWmwVnmgcdh4b8phaUcQ3LAC9K"

In [23]:
conn = sqlite3.connect('twitter_AMZN.db')

c = conn.cursor()

def create_table():
    c.execute("CREATE TABLE IF NOT EXISTS sentiment(unix REAL, tweet TEXT, sentiment REAL)")
    conn.commit()

create_table()

In [24]:
word_to_index = imdb.get_word_index()
loaded_model = load_model('GRU_all_vocab_model.h5')

def sentiment(tweet):
    word_token = word_tokenize(tweet)
    target_text = []
    for word in word_token:
        # 단어를 소문자로 변환 => 만약에 영화리뷰 감정 분석에 쓰였던 단어면 포함
        # 아니면 제외
        if word.lower() in word_to_index:
            # 단어를 word_to_index에서 찾아서, 숫자로 변환 
#             if word_to_index[word.lower()] < 10000:
            target_text.append(word_to_index[word.lower()])
    if len(target_text) == 0:
        return None # 분석할 단어가 없어서 감정 분석 불가능
    
    print(len(target_text))
    x_text = np.array([target_text])
    
    max_len = 500
    X_train = pad_sequences(x_text, maxlen = max_len)
    
    try:
        predicted = loaded_model.predict(X_train)
    except:
        return None

    return predicted

In [27]:
c.execute("DELETE FROM sentiment")
conn.commit()

In [ ]:
class listener(StreamListener):
    def on_data(self, data):
        # twitter에서 업데이트 된 twit이 data로 저장.
        # 업데이트 된 twit의 text가 긍정인지, 부정인지를 판단
        # 각 twit이 긍정이었는지, 부정이었는지 저장. 
        # 위 데이터가 시간대별로 쌓이면 => 어떤 주식 sentiment가 어떻게 변했는지 그리고 어떻게 변화하고 있는지.
        # json
        all_data = json.loads(data)
        
        tweet = all_data["text"]
        sentiment_value = sentiment(tweet)
        
        time_ms = all_data['timestamp_ms']
        print(sentiment_value)
        if sentiment_value != None:
            c.execute("INSERT INTO sentiment (unix, tweet, sentiment) VALUES (?,?,?)", (time_ms, tweet, float(sentiment_value[0][0])))
            conn.commit()
            output = open("twitter-AMZN.txt", "a")
            output.write(str(sentiment_value[0][0]))
            output.write('\n')
            output.close()
        return True
    def on_error(self, status):
        print(status)

auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

twitter_stream = Stream(auth, listener())
twitter_stream.filter(track=["AMZN"])

5
[[0.51338655]]
14
[[0.06252927]]
None
8
[[0.05894029]]
3
[[0.5207667]]
1
[[0.69687325]]
7
[[0.00107187]]
14
[[0.26335812]]
1
[[0.69687325]]
1
[[0.69687325]]
8
[[0.00591773]]
1
[[0.83939147]]
2
[[0.3193618]]
1
[[0.6759518]]
None
None
14
[[0.00689328]]
None
7
[[0.00107187]]
None
11
[[0.99977136]]
1
[[0.69687325]]
6
[[0.56432456]]
None
1
[[0.69687325]]
19
[[0.00031766]]
2
[[0.00932908]]
None
None
2
[[0.00146899]]
2
[[0.24198082]]
4
[[0.8804359]]
21
[[0.00392577]]
8
[[0.9988778]]
None
5
[[0.01864481]]
1
[[0.994987]]
20
[[0.9995495]]
6
[[0.03016105]]
1
[[0.09288439]]
6
[[0.01131618]]
6
[[0.03016105]]
None
None
16
[[0.993305]]
None
1
[[0.77414393]]
8
[[0.06974643]]
8
[[0.00798929]]
4
[[0.00172514]]
None
7
[[0.00209439]]
1
[[0.09288439]]
4
[[0.43496907]]
10
[[0.635687]]
4
[[0.0001469]]
20
[[0.9969214]]
16
[[0.9727864]]
None
10
[[0.01173526]]
18
[[0.03670701]]
3
[[0.99071395]]
None
14
[[8.685121e-06]]
1
[[0.73091555]]
None
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
24
[[0.94415796]]


2
[[0.49322197]]
3
[[0.08706501]]
None
12
[[0.00398606]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
11
[[0.01072496]]
1
[[0.69687325]]
22
[[0.0001345]]
5
[[0.00176376]]
1
[[0.69687325]]
13
[[0.00010913]]
None
13
[[0.00032294]]
1
[[0.69687325]]
1
[[0.7776425]]
1
[[0.96050876]]
1
[[0.9999914]]
1
[[0.69687325]]
9
[[0.6640807]]
17
[[0.09528068]]
7
[[0.5345108]]
None
None
1
[[0.69687325]]
13
[[0.98845005]]
10
[[0.9813909]]
19
[[0.00224906]]
1
[[0.7776425]]
None
1
[[0.69687325]]
None
4
[[0.76148754]]
None
5
[[0.00176376]]
1
[[0.69687325]]
8
[[0.4599786]]
None
19
[[0.00224906]]
6
[[0.7131381]]
7
[[0.9999782]]
3
[[0.24667856]]
1
[[0.69687325]]
None
20
[[0.00590184]]
11
[[0.80776566]]
15
[[0.68959606]]
None
2
[[0.9656126]]
1
[[0.69687325]]
18
[[0.03670701]]
1
[[0.69687325]]
1
[[0.09288439]]
4
[[0.48466817]]
1
[[0.69687325]]
4
[[0.99161804]]
1
[[0.69687325]]
10
[[0.967329]]
1
[[0.95403564]]
18
[[0.03670701]]
13
[[0.02275723]]
None
5
[[0.00176376]]
1
[[0.69687325]]
2
[[0.40209502]]
None
3

None
1
[[0.69687325]]
11
[[0.99392104]]
None
1
[[0.69687325]]
3
[[0.00547174]]
12
[[0.42775208]]
1
[[0.69687325]]
None
20
[[0.9843235]]
1
[[0.69687325]]
1
[[0.69687325]]
19
[[0.00224906]]
1
[[0.69687325]]
1
[[0.99170935]]
1
[[0.09288439]]
1
[[0.69687325]]
None
18
[[0.03670701]]
3
[[0.06170821]]
17
[[0.11263198]]
1
[[0.69687325]]
13
[[0.00012434]]
1
[[0.69687325]]
27
[[6.257696e-05]]
2
[[0.5042785]]
20
[[0.0036315]]
1
[[0.09288439]]
None
7
[[1.7595265e-05]]
1
[[0.99747336]]
1
[[0.69687325]]
1
[[0.69687325]]
20
[[0.0036315]]
1
[[0.69687325]]
1
[[0.69687325]]
3
[[0.9762217]]
1
[[0.69687325]]
2
[[0.00932908]]
1
[[0.69687325]]
1
[[0.69687325]]
2
[[0.9692744]]
5
[[0.27886182]]
None
None
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
18
[[0.00331768]]
None
1
[[0.69687325]]
None
6
[[0.00910738]]
11
[[0.16117316]]
1
[[0.69687325]]
1
[[0.69687325]]
6
[[1.5979274e-05]]
None
None
None
2
[[0.15050116]]
1
[[0.69687325]]
None
1
[[0.69687325]]
13
[[0.0011678]]
None
7
[[0.997058]]


11
[[0.00011247]]
5
[[0.9423191]]
None
1
[[0.69687325]]
1
[[0.69687325]]
11
[[0.00011247]]
11
[[0.00366741]]
None
16
[[0.03963834]]
3
[[0.54455006]]
2
[[0.00712663]]
10
[[0.04799265]]
None
5
[[0.5646064]]
None
1
[[0.69687325]]
16
[[0.09102646]]
18
[[0.03670701]]
None
2
[[0.9987434]]
None
3
[[0.8559171]]
1
[[0.69687325]]
15
[[0.95275164]]
1
[[0.69687325]]
16
[[4.5218578e-05]]
1
[[0.69687325]]
3
[[0.01849192]]
15
[[0.00096452]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
None
14
[[0.33361995]]
19
[[0.00365829]]
4
[[0.00028101]]
18
[[0.03670701]]
1
[[0.69687325]]
3
[[0.95800686]]
1
[[0.69687325]]
10
[[0.00019515]]
2
[[0.42577326]]
None
7
[[0.55158323]]
7
[[4.4458044e-05]]
12
[[0.00124311]]
1
[[0.69687325]]
19
[[0.00224906]]
4
[[0.9824616]]
1
[[0.9496697]]
8
[[0.03070486]]
9
[[0.00129959]]
None
1
[[0.09288439]]
18
[[0.03670701]]
1
[[0.69687325]]
None
1
[[0.45154387]]
3
[[0.00025064]]
1
[[0.69687325]]
7
[[0.99565744]]
18
[[0.03670701]]
1
[[0.80561]]
1
[[0.69687325]]
2
[[0.3210268]]
2

17
[[0.08418462]]
1
[[0.69687325]]
None
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
2
[[0.5000617]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
6
[[0.00191221]]
1
[[0.69687325]]
1
[[0.69687325]]
4
[[0.99997187]]
3
[[0.00222215]]
3
[[0.99604905]]
1
[[0.69687325]]
9
[[0.3480387]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
11
[[0.00011247]]
19
[[0.00224906]]
19
[[0.97412556]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
None
7
[[0.00367859]]
1
[[0.69687325]]
19
[[0.00224906]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
7
[[0.01244119]]
11
[[0.00011247]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
None
None
13
[[0.03591514]]
9
[[0.13401836]]
1
[[0.69687325]]
1
[[0.69687325]]
19
[[0.00224906]]
1
[[0.69687325]]
15
[[0.01197895]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
3
[[0.9762217]]
1
[[0.69687325]]
8
[[0

1
[[0.69687325]]
1
[[0.69687325]]
19
[[0.10840219]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
5
[[0.91195524]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
9
[[0.00129959]]
1
[[0.69687325]]
1
[[0.69687325]]
15
[[0.7826493]]
15
[[0.03322354]]
1
[[0.69687325]]
1
[[0.69687325]]
None
14
[[0.00039768]]
1
[[0.69687325]]
None
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.98399186]]
1
[[0.69687325]]
None
None
1
[[0.69687325]]
1
[[0.69687325]]
20
[[0.9843235]]
4
[[0.00976011]]
1
[[0.9717747]]
5
[[0.5239077]]
7
[[0.6502805]]
11
[[4.7241243e-05]]
None
1
[[0.69687325]]
6
[[0.92598313]]
1
[[0.09288439]]
1
[[0.69687325]]
1
[[0.95403564]]
1
[[0.69687325]]
14
[[0.0343827]]
1
[[0.69687325]]
19
[[0.79767907]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
17
[[0.08418462]]
None
1
[[0.69687325]]
7
[[0.00367859]]
9
[[0.00580549]]
1
[[0.69687325]]
1
[[0.69687325]]
5
[[0.86514103]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
2
[[0.00932908]]
1
[[0.696873

[[0.69687325]]
12
[[0.96349573]]
7
[[0.05614236]]
8
[[0.94076645]]
1
[[0.69687325]]
3
[[0.09312436]]
1
[[0.69687325]]
1
[[0.69687325]]
None
16
[[1.4136649e-06]]
1
[[0.69687325]]
1
[[0.09288439]]
1
[[0.69687325]]
1
[[0.69687325]]
19
[[0.19436172]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
4
[[0.00081748]]
1
[[0.69687325]]
5
[[0.51338655]]
None
15
[[0.01702037]]
16
[[0.04126158]]
None
None
14
[[0.00039768]]
11
[[0.8611298]]
1
[[0.69687325]]
8
[[0.00463283]]
16
[[0.01570877]]
4
[[0.04003036]]
None
19
[[0.00042415]]
21
[[0.9982585]]
1
[[0.98855877]]
5
[[0.12544855]]
20
[[0.9921895]]
17
[[0.05318981]]
None
22
[[0.9808086]]
5
[[0.7594423]]
5
[[0.99670124]]
5
[[0.4212786]]
3
[[0.5334672]]
None
None
None
1
[[0.69687325]]
None
6
[[0.9998437]]
6
[[0.01938605]]
1
[[0.69687325]]
1
[[0.09288439]]
1
[[0.69687325]]
9
[[0.24831343]]
1
[[0.69687325]]
None
None
1
[[0.69687325]]
1
[[0.9990964]]
11
[[0.50693727]]
14
[[0.99531007]]
2
[[0.34689137]]
1
[[0.69687325]]
None
None
5
[[0.13048902]]
4
[[0

2
[[0.07092041]]
7
[[0.32188815]]
2
[[0.9995322]]
1
[[0.69687325]]
None
22
[[0.9683236]]
10
[[0.01581877]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
3
[[0.2181195]]
8
[[0.21844834]]
1
[[0.69687325]]
1
[[0.69687325]]
2
[[0.01663008]]
2
[[0.06972736]]
None
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
None
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
4
[[0.28848502]]
11
[[0.11181599]]
None
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
3
[[0.785189]]
1
[[0.69687325]]
7
[[0.00107187]]
18
[[0.7265488]]
1
[[0.69687325]]
10
[[0.9850621]]
2
[[0.9043591]]
3
[[0.08038855]]
None
12
[[0.7894975]]
1
[[0.69687325]]
None
3
[[0.39061442]]
3
[[0.00072393]]
None
11
[[0.3467651]]
1
[[0.69687325]]
None
1
[[0.69687325]]
3
[[0.75101346]]
20
[[0.5720684]]
None
8
[[0.99457234]]
1
[[0.69687325]]
16
[[0.75891167]]
6
[[0.9273582]]
None
12
[[0.5418078]]
1

[[0.69687325]]
15
[[0.0433577]]
14
[[0.00073764]]
None
3
[[0.44206828]]
10
[[0.995232]]
None
1
[[0.69687325]]
7
[[0.02557555]]
None
1
[[0.69687325]]
1
[[0.69687325]]
22
[[0.9914899]]
4
[[0.43586078]]
None
17
[[0.02521759]]
None
None
1
[[0.69687325]]
1
[[0.69687325]]
None
11
[[0.4209383]]
11
[[0.4209383]]
11
[[0.4209383]]
11
[[0.4209383]]
14
[[0.33361995]]
11
[[0.4209383]]
11
[[0.4209383]]
11
[[0.4209383]]
2
[[0.9925384]]
1
[[0.69687325]]
11
[[0.4209383]]
None
1
[[0.69687325]]
18
[[0.00153247]]
None
1
[[0.69687325]]
1
[[0.69687325]]
9
[[0.04790729]]
13
[[0.0011678]]
None
8
[[0.00229335]]
1
[[0.69687325]]
9
[[0.99771833]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.69687325]]
17
[[0.02521759]]
7
[[0.00525627]]
23
[[0.00182524]]
11
[[0.00011247]]
2
[[0.05541331]]
3
[[0.04967603]]
None
2
[[0.5042785]]
None
21
[[0.0063172]]
15
[[0.09225279]]
None
7
[[0.99565744]]
17
[[0.02521759]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.09288439]]
1
[[0.69687325]]
1
[[0.69687325]]
1
[[0.6968732